In [57]:
import json
from datetime import timedelta

with open("/content/speech.json", encoding='utf-8', errors='ignore') as json_data:
     data = json.load(json_data, strict=False)

In [58]:
print (data)

{'AudioFileResults': [{'AudioFileName': 'Channel.0.wav', 'AudioFileUrl': None, 'AudioLengthInSeconds': 290.31, 'CombinedResults': [{'ChannelNumber': None, 'Lexical': "they will call good holder services my name is jimmy only had the card number please yes but first let me ask my daughter receive this gift card for christmas southwest bank gift card and we're trying to get a balance on it keeps asking for a pin with course she doesn't have a pen so can you switch the balances i'm sorry about it but uh if you want to check the balance as soon as we service the the card 'cause we service model credit union can i get the card number please for me to check yes it's a it's a gift card OK OK i'm now loading it here in our system one moment please in your name is you are i'm sorry for the delay just said still getting the information OK uhm here's something came up now this system and showing here that this card is broadway uh nothing back from this card is showing zero balance zero this actio

In [ ]:
audioFileResults = data['AudioFileResults']
speakers_timestamps = {}
speakers = []

for i in audioFileResults:
        
    seg_results = i['SegmentResults']
    
    #find the speakers
    for j in seg_results:
        if j['SpeakerId'] not in speakers:
            speakers.append(j['SpeakerId'])            
    print("Unique set of speakers :")
    print(speakers)
    print('\n')
    
    for speaker in speakers:
        speaker_dict = {'speakerId' : speaker}
        OffsetInSeconds = []
        EndTimeInSeconds = []
        for segmentResult in seg_results:
            if segmentResult['SpeakerId'] == speaker:
                
                OffsetInSeconds.append(segmentResult['OffsetInSeconds'])
                #compute end time
                endTime = (timedelta(seconds=segmentResult['OffsetInSeconds']) + \
                timedelta(seconds=segmentResult['DurationInSeconds'])).total_seconds()
                EndTimeInSeconds.append(endTime)
               
        speaker_dict['start_timestamps'] = OffsetInSeconds
        speaker_dict['end_timestamps'] = EndTimeInSeconds
        
        speakers_timestamps[speaker] = speaker_dict
    
    print(speakers_timestamps)
    print('\n')

#convert to json object
json_obj = json.dumps(speakers_timestamps, indent=4)

print("printing json object.....\n")
print(json_obj)

with open("audiotimestamp.json", "w") as outfile: 
    outfile.write(json_obj)

In [63]:
!pip install pydub
import os
from pydub import AudioSegment

for speaker in speakers:
  element = speakers_timestamps[speaker]

  for i in range(len(element['start_timestamps'])):
    newAudio = AudioSegment.from_wav("/content/sample_data/Samoy.wav")
    newAudio = newAudio[int(element['start_timestamps'][i]*1000):int(element['end_timestamps'][i]*1000)]
    newAudio.export('newSong' + str(i) + '.wav', format="wav")

  combined_sounds = AudioSegment.from_wav("/content/newSong0.wav")
  
  for i in range(len(element['start_timestamps'])):
    sound = AudioSegment.from_wav("/content/newSong" + str(i) + ".wav")
    combined_sounds = combined_sounds + sound
    combined_sounds.export('newClip' + element['speakerId'] + '.wav', format="wav")

  for i in range(len(element['start_timestamps'])):
    os.remove('/content/newSong' + str(i) + '.wav')